https://docs.google.com/document/d/1mrb-bTipyDp_c5Re2QUKKxb7vw0EBqy7Ux938JGv0hw/edit\
https://web.stanford.edu/class/archive/cs/cs224n/cs224n.1174/reports/2760332.pdf

http://pytorch.org/tutorials/intermediate/char_rnn_generation_tutorial.html

In [6]:
import glove
import nltk
import numpy as np

import load

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

cuda = False

# Charger les donné

In [7]:
data = load.load_jokes()

## preprocessing

In [8]:
X = []
for i in range(len(data)):
    X += [load.clean_words(data[i])]
    

# Glove ou Word2vec

In [10]:
def str_to_float(val):
    try:
        return float(val)
    except:
        return 0.0

def loadGloveModel(gloveFile):
    print("Loading Glove Model")
    f = open(gloveFile,'r')
    model = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
            
        embedding = np.array([str_to_float(val) for val in splitLine[1:]])
        model[word] = embedding
    print("Done.",len(model)," words loaded!")
    f.close()
    return model

def moy_sent(sentence, glove):
    sent = np.array([glove[w] for w in sentence])
    return np.mean(sent)
glove = loadGloveModel("./glove.840B.300d.txt")

for w, v in glove.items():
    if len(v) < 300:
        glove[w] = np.concatenate((v, np.zeros(300-len(v))))
    elif len(v) > 300:
        glove[w] = v[:300]

def to_glove(X):
    res = np.empty((len(X), 300))
    for i in range(len(X)):
        try :
            res[i] = glove[X[i]][:300]
        except KeyError:
            print(X[i])
            res[i] = np.zeros(300)
    return res

Loading Glove Model
Done. 2195884  words loaded!


In [11]:
X_tmp = []
for i in range(len(X)):
    sent = []
    for j in range(len(X[i])):
        try:
            a = glove[X[i][j]]
            sent += [X[i][j]]
        except KeyError:
            print(X[i][j])
            pass
    X_tmp += [sent]
    
X = X_tmp

half-
natural–
9%
25%
'Insane
Posse'
parents'
'Entourage.'
$50,000
streets'
‘I
glad.'
sleep-reading
Self-Recrimination
YOU'RE
I'M
50%
$7
Geostorm
$100
98%
100%
Jesus'
$500
Nine-Nine-Nine
$1
90%
MySpace—but
Solowitz
breasturant
marijuana-themed
pot-themed
Unpublishable”
Judy"-style
swimmer—
interview—
ennu-ey
$92
$50
$25
$2.5
‘Men.'
I’d
You’d
70%
MWA-ha-ha-ha…
Learnin’
fence-jumping
you'd
Porn—
Olympics'
Trump-Russia
lovin'
#1
#1
he'd
‘hard'
47%
they’d
day-drinking
$75,000
bonin.'
$3
b*****
FIIIINE
1.6%
Hey—it
Hey—
Suckapalooza
one-foot-tall
peppermint-flavored
'Tis
'tis
p****
Vamanaos
dollars'
man-bun
stank"
$80
$4.95
was—
$199
$200
won"
'clusterf***.'
#1
super-awkward
60%
40%
players'
office…
indoors—
d***
leg…
$8,000
kids'
Girls'
self-pumping
10%
100%
means—he
me—
Deflategate
is—when
62-gun
they'd
88%
owners'
$40,000
pay…
Frankly—
belieb
assemble-yourself
Hmmm…
70%
30%
$200
10%
10%
10%
‘90
Vegas…
there—
$20
(accidentally
router)
$50,000
J"—for
‘J'
36%
sorry—I
30%
50%
100%
friends'
75

#Alibi
50%
he'd
$76
problems…
say—
it—

Well-
it—

$60,000
‘80s
butt-dialed
f***
70%
Aaaay
Brexit
PEOPLE-WHO-LIVE-IN-ENGLAND
9%
#1
#2
$1,500
'yes.'
$500
15%
before..
2,600-year-old
‘It
Men.'
$185
$25
lowest-earning
'how
surgery.'
you—
Hey—my
$3.99
$200
who'd
he’d
Mall-walking
winner—Kim
40%
home—we
riot…
$500
$500.00
crazy­-tight
half-alligator
sh**
McWhopper
Ba-lack
$450,000
McBites,"-
‘Fish
McBite,'
McHear
sense—
$2
$2
parents’
Flamin’
kiss-cam
'The
round.''
1%
35%
‘this’
'following
law.'
explanation—
$70
he'd
'Judge
Judy.'
MEEEE!!!
$50,000
Handsometown
$300
Wait…
'go
somewhere?'
$380
Jesus'
#1
$800
extra-extra-large
Kim-Jong
Carsony
$9.99
$14.99
#30
#1
wristwatch-like
53%
47%
Dodgers-Astros
30%
‘open.'
'sup
Tower"—it
'acting
10-year-old.'
bi-
10%
wait…
$100
sh**ty
Ladies’
800%
since…
know…
Egypt—The
smoking—quit
30%
30%
Nanoc
backwards”
here—I
'murder.'
$2
man-opotamus
$600,000
$30
$800
Underreacters
true-
'bye'
Asian-on-Asian
Mampon
$10,000
$200
$250
$25
Breasts”
users'
conan@obrie

In [7]:
def knn(glove, word):
    mot = None
    dist_min = 90000
    for w, v in glove.items():
        dist = np.linalg.norm(v[:300]-word)
        if dist < dist_min:
            mot = w
            dist_min = dist
    return mot

## attention

In [9]:
def attention(sentence, glove):
    try:
        pos_tag = nltk.pos_tag(sentence)
    except IndexError :
        print(sentence)
    
    sent = to_glove(sentence)

    
    masque = np.array([1 if (t == 'NN' or t == 'NNP') else 0 for _,t in pos_tag])
    sent = np.array(sent[masque==1], dtype="float64")
    if np.sum(masque) == 0:
        return np.zeros(300)
    return np.array(np.mean(sent, axis=0))

#  Decoder (LSTM)

In [10]:
class LSTMWord(nn.Module):

    def __init__(self, taille_vecteur, hidden_dim, num_layers):
        super(LSTMWord, self).__init__()
        self.taille_vecteur = taille_vecteur
        self.hidden_dim = hidden_dim
        
        self.num_layers = num_layers
       
        self.lstm = nn.LSTM(taille_vecteur*2, hidden_dim, num_layers=num_layers)
        self.hidden = self.init_hidden()
        
        self.lin = nn.Linear(hidden_dim , taille_vecteur)
#         self.softmax = nn.LogSoftmax(dim=1) regression  pas sur
        

    def init_hidden(self):
        if cuda:
            return (autograd.Variable(torch.zeros(self.num_layers, 1, self.hidden_dim).cuda()),
                    autograd.Variable(torch.zeros(self.num_layers, 1, self.hidden_dim).cuda()))
        else:
            return (autograd.Variable(torch.zeros(self.num_layers, 1, self.hidden_dim)),
                    autograd.Variable(torch.zeros(self.num_layers, 1, self.hidden_dim)))


    def forward(self, inputs, topic=None):
        if topic is None:
            topic = autograd.Variable(torch.zeros(1, 1, self.taille_vecteur))
    
        
        inputs = torch.cat((inputs, topic)).view(1, -1)
        out, self.hidden = self.lstm(inputs, self.hidden)
        out = self.lin(out)
        return out

## train

In [11]:
def is_stop(mot, stop):
    for w in stop:
        dist = np.linalg.norm(mot-w)
        
        if dist < 1:
            return True
    return False

In [12]:
def train_epoch(X, model, hidden_dim=10, num_layers=1, glove=None):
    if cuda:
        model.cuda()
    
    optimizer = optim.Adam(model.parameters())
    if cuda:
        loss_function = nn.MSELoss().cuda()
    else:
        loss_function = nn.MSELoss()
    
    loss_cum = 0
    acc = 0
    stop_word = [glove["."], glove["?"], glove["!"]]
    
    for j in range(len(X)):

        model.zero_grad()
        model.hidden = model.init_hidden()
        glov_X = to_glove(X[j])
        if cuda:
            topic = autograd.Variable(torch.FloatTensor(attention(X[j], glove)).cuda())
            tens_X = autograd.Variable(torch.FloatTensor(glov_X).cuda())
        else:
            topic = autograd.Variable(torch.FloatTensor(attention(X[j], glove)))
            tens_X = autograd.Variable(torch.FloatTensor(glov_X))
            
        output =[model(tens_X[0], topic)]
        m=1
        sum_=output[-1]
#         
        for i in range(1, len(tens_X)):
            output += [model(output[i-1].view(300), topic)]
            m+=1
            sum_+=output[-1]
            
            if cuda:
                if is_stop(output[-1].data.cpu().numpy().reshape(300), stop_word):
                    loss = loss_function(output[-1], tens_X[i])

                    loss_cum += loss.data.cpu().numpy()
                    
                    break
            else:
                if is_stop(output[-1].data.numpy().reshape(300), stop_word):
                    loss = loss_function(output[-1], tens_X[i])

                    loss_cum += loss.data.numpy()
                    break

            loss = loss_function(output[-1], tens_X[i])
        
            if cuda:
                 loss_cum += loss.data.cpu().numpy()
            else:
                
                loss_cum += loss.data.numpy()
            
        loss.backward()
        optimizer.step()

    return acc/len(X), loss_cum/len(X)

def train(X, epoch, hidden_dim=10, num_layers=1, glove=None):
    #glove
    model = LSTMWord(len(glove[X[0][1]]), hidden_dim, num_layers)
    for i in range(epoch):
        print("epoch", i)
        acc , loss = train_epoch(X, model, hidden_dim, num_layers, glove=glove)
        print("acc : {}, loss : {}".format(acc, loss))
    return model


In [13]:
epochs = 1
hidd_dim = 50
layers = 10
model = train(X, epochs, hidd_dim, layers, glove=glove)

epoch 0
acc : 0.0, loss : [ 5.95432854]


# Generateur

In [19]:
def gen(X, model, glove, taille_max=30):
    """
    x chaque mot de debut de phrase
    """
    out = []
    stop_word = [glove["."], glove["?"], glove["!"]]
    
    all_blague = []
    for j in range(len(X)):

        model.zero_grad()
        model.hidden = model.init_hidden()
        glov_X = to_glove(X[j])

        if cuda:
            topic = autograd.Variable(torch.FloatTensor(attention(X[j], glove)).cuda())
            tens_X = autograd.Variable(torch.FloatTensor(glov_X).cuda())
        else:
            topic = autograd.Variable(torch.FloatTensor(attention(X[j], glove)))
            tens_X = autograd.Variable(torch.FloatTensor(glov_X))
            
        print(knn(glove, tens_X[0].view(300).data.numpy()), end=" ")
        output =[model(tens_X[0], topic)]

        m=1
        sum_=output[-1]
        dist = np.linalg.norm(output[-1].view(300).data.numpy()-glove["."])
        print(knn(glove, output[-1].view(300).data.numpy()), end=" ")
        for i in range(1, 30):
            output += [model(output[i-1].view(300), topic)]
            m+=1
            sum_+=output[-1]
             print(dist)
            print(knn(glove, output[-1].view(300).data.numpy()), end=" ")
            
            if cuda:
                if is_stop(output[-1].data.cpu().numpy().reshape(300), stop_word):
                    break
            else:
                if is_stop(output[-1].data.numpy().reshape(300), stop_word):
                    print(knn(glove, output[-1].view(300).data.numpy()))
            
        all_blague += [output]

    return all_blague

In [20]:
def blague(blague_gen):
    res = []
    for i in range(len(blague_gen)):
        phrase = []
        sent = blague_gen[i]
        for j in range(len(sent)):
            print(sent[j].view(300).data.cpu().numpy())
            phrase += [knn(glove, sent[j].view(300).data.cpu().numpy())]
            
        print(phrase)
        res += [phrase]
    return res            

In [21]:
idx = np.arange(len(X))
np.random.shuffle(idx)
idx = idx[:500]

In [ ]:
blague_gen = gen(np.array(X)[idx], model, glove)
# knn(glove, blague_gen[0][0].view(300).data.numpy())

Over . . . .
. 

In [18]:
blague(blague_gen)
del blague_gen

TypeError: object of type 'numpy.float32' has no len()